# README
* EfficientNet-B6
* Train only kernel
* Fine Tuning
* Ben's preprocessing

# 1. Setup

In [1]:
!pip install torch==1.1.0

     |████████████████████████████████| 676.9MB 66kB/s 
ERROR: allennlp 0.8.4 requires awscli>=1.11.91, which is not installed.
ERROR: allennlp 0.8.4 requires flaky, which is not installed.
ERROR: allennlp 0.8.4 requires responses>=0.7, which is not installed.
  Found existing installation: torch 1.2.0
    Uninstalling torch-1.2.0:
      Successfully uninstalled torch-1.2.0


In [2]:
!pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ../input/nvidiaapex/repository/NVIDIA-apex-39e153a

/opt/conda/lib/python3.6/site-packages/pip/_internal/commands/install.py:243: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-1kjvlf7j
Created temporary directory: /tmp/pip-req-tracker-olk7iz9l
Created requirements tracker '/tmp/pip-req-tracker-olk7iz9l'
Created temporary directory: /tmp/pip-install-u4n6c2h2
Processing /kaggle/input/nvidiaapex/repository/NVIDIA-apex-39e153a
  Created temporary directory: /tmp/pip-req-build-jpyevgqf
  Added file:///kaggle/input/nvidiaapex/repository/NVIDIA-apex-39e153a to build tracker '/tmp/pip-req-tracker-olk7iz9l'
    Running setup.py (path:/tmp/pip-req-build-jpyevgqf/setup.py) egg_info for package from file:///kaggle/input/nvidiaapex/repository/NVIDIA-apex-39e153a
    Running command python setup.py egg_info
    torch.__version__  =  1.1.0
    running egg_info
    creating pip-egg

In [3]:
import cv2
import matplotlib.pyplot as plt
from os.path import isfile
import torch.nn.init as init
import torch
import torch.nn as nn
import numpy as np
import pandas as pd 
import os
from PIL import Image, ImageFilter
print(os.listdir("../input"))
from sklearn.model_selection import train_test_split, StratifiedKFold
from torch.utils.data import Dataset
from torchvision import transforms
import torch.optim as optim
import time
from tqdm import tqdm
from torch.autograd import Variable
import torch.functional as F
from tqdm import tqdm
from sklearn import metrics
import urllib
import pickle
import cv2
import torch.nn.functional as F
from torchvision import models
import seaborn as sns
import random
from apex import amp
import sys

%matplotlib inline

['nvidiaapex', 'aptos2019-blindness-detection', 'efficientnet', 'efficientnet-pytorch']


Loading Pre-trained EfficientNet-B6 model and Weight

In [4]:
package_path = '../input/efficientnet/efficientnet-pytorch/EfficientNet-PyTorch/'
sys.path.append(package_path)

from efficientnet_pytorch import EfficientNet
model = EfficientNet.from_name('efficientnet-b6')
model.load_state_dict(torch.load('../input/efficientnet-pytorch/efficientnet-b6-c76e70fd.pth'))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

Seed setting

In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [6]:
seed_everything(1234)
num_classes = 1
IMG_SIZE = 256
lr = 1e-3
batch_size = 32

In [7]:
# DataFrame for Train / Test set
df = pd.read_csv('../input/aptos2019-blindness-detection/train.csv')
df_test = pd.read_csv('../input/aptos2019-blindness-detection/test.csv')
df_sample = pd.read_csv('../input/aptos2019-blindness-detection/sample_submission.csv')

y_train = df['diagnosis']

# Image data
train = '../input/aptos2019-blindness-detection/train_images/'
test = '../input/aptos2019-blindness-detection/test_images/'

In [8]:
# Split train / valid set
df_train, df_valid = train_test_split(df, test_size=0.1, random_state=1234, stratify=df.diagnosis)

df_train.reset_index(drop=True, inplace=True)
df_valid.reset_index(drop=True, inplace=True)

print('df_train :', df_train.shape)
print('df_valid :', df_valid.shape)

df_train : (3295, 2)
df_valid : (367, 2)


# 2. Image Preprocessing

In [9]:
def expand_path(p):
    p = str(p)
    if isfile(train + p + ".png"):
        return train + (p + ".png")
    if isfile(train_2015 + p + '.png'):
        return train_2015 + (p + ".png")
    if isfile(test + p + ".png"):
        return test + (p + ".png")
    return p

In [10]:
def crop_image_from_gray(img,tol=7):
    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>tol
        
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): # image is too dark so that we crop out everything,
            return img # return original image
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
    #         print(img1.shape,img2.shape,img3.shape)
            img = np.stack([img1,img2,img3],axis=-1)
    #         print(img.shape)
        return img

# 3. Dataset & DataLoader setting

In [11]:
class MyDataset(Dataset):
    
    def __init__(self, dataframe, transform=None):
        self.df = dataframe
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        label = self.df.diagnosis.values[idx]
        label = np.expand_dims(label, -1)
        p = self.df.id_code.values[idx]
        p_path = expand_path(p)
        image = cv2.imread(p_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = crop_image_from_gray(image)
        image = cv2.resize(image, (256, 256))
        image = cv2.addWeighted(image, 4, cv2.GaussianBlur(image, (0,0), 30), -4, 128)
        image = transforms.ToPILImage()(image)
        
        if self.transform:
            image = self.transform(image)
            
        return image, label

In [12]:
# Train loader
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation((-120, 120)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

trainset = MyDataset(df_train, transform=train_transform)

train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=4)

# Valid loader
validset = MyDataset(df_valid, transform=train_transform)

valid_loader = torch.utils.data.DataLoader(validset, batch_size=batch_size, shuffle=False, num_workers=4)

# 4. Define Network

In [13]:
in_features = model._fc.in_features
model._fc = nn.Linear(in_features=in_features, out_features=num_classes)
model.cuda()

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 56, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(56, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        56, 56, kernel_size=(3, 3), stride=[1, 1], groups=56, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(56, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        56, 14, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        14, 56, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        56, 32, kernel_siz

In [14]:
# Loss function
criterion = nn.MSELoss() # as a regression model

# Optimizer + Weight decay
optimizer = optim.Adam(params=model.parameters(), lr=lr, weight_decay=1e-5)

# Learning rate scheduling
# scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[num_epochs*0.5, num_epochs*0.8], gamma=0.1)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

# Amp initilize
model, optimizer = amp.initialize(model, optimizer, opt_level="O1",verbosity=0)

In [15]:
def train_model():
    model.train() 
        
    avg_loss = 0.
    optimizer.zero_grad()
    for idx, (imgs, labels) in enumerate(tqdm(train_loader)):
        imgs_train, labels_train = imgs.cuda(), labels.float().cuda()
        output_train = model(imgs_train)
        loss = criterion(output_train,labels_train)
        with amp.scale_loss(loss, optimizer) as scaled_loss:
            scaled_loss.backward()
        optimizer.step() 
        optimizer.zero_grad() 
        avg_loss += loss.item() / len(train_loader)
        
    return avg_loss

def valid_model():
    
    avg_val_loss = 0.
    model.eval()
    with torch.no_grad():
        for idx, (imgs, labels) in enumerate(tqdm(valid_loader)):
            imgs_vaild, labels_vaild = imgs.cuda(), labels.float().cuda()
            output_test = model(imgs_vaild)
            avg_val_loss += criterion(output_test, labels_vaild).item() / len(valid_loader)
        
    return avg_val_loss

# 5. Training & Validation

In [16]:
best_avg_loss = 100.0
n_epochs      = 20

for epoch in range(n_epochs):
    
    print('lr:', scheduler.get_lr()[0]) 
    start_time   = time.time()
    avg_loss     = train_model()
    avg_val_loss = valid_model()
    elapsed_time = time.time() - start_time 
    print('Epoch {}/{} \t loss={:.4f} \t val_loss={:.4f} \t time={:.2f}s'.format(
        epoch + 1, n_epochs, avg_loss, avg_val_loss, elapsed_time))
    
    if avg_val_loss < best_avg_loss:
        best_avg_loss = avg_val_loss
        torch.save(model.state_dict(), 'b5_weight_best.pt')
    
    scheduler.step()

  0%|          | 0/103 [00:00<?, ?it/s]

lr: 0.001


100%|██████████| 12/12 [01:06<00:00,  3.32s/it]


Epoch 1/20 	 loss=0.7492 	 val_loss=0.9279 	 time=665.26s


  0%|          | 0/103 [00:00<?, ?it/s]

...

Epoch 18/20 	 loss=0.1600 	 val_loss=0.3283 	 time=651.49s
lr: 1.0000000000000002e-06


  0%|          | 0/103 [00:00<?, ?it/s]

Epoch 19/20 	 loss=0.1533 	 val_loss=0.3340 	 time=653.75s
lr: 1.0000000000000002e-06


100%|██████████| 12/12 [01:05<00:00,  3.21s/it]

Epoch 20/20 	 loss=0.1582 	 val_loss=0.3311 	 time=652.69s
